In [67]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

#

from sklearn import tree

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

In [68]:
metadata_df = cfs.retrieve_metadata_df()

In [69]:
red_route_gmID_list  = metadata_df[ metadata_df[ 'Route' ] == 'Red' ][ 'groupMetadataID' ].tolist()

green_route_gmID_list  = metadata_df[ metadata_df[ 'Route' ] == 'Green' ][ 'groupMetadataID' ].tolist() 

blue_route_gmID_list  = metadata_df[ metadata_df[ 'Route' ] == 'Blue' ][ 'groupMetadataID' ].tolist()

In [70]:
green_route_training_gmID_list = green_route_gmID_list[ : len( green_route_gmID_list) // 2 ]

green_route_testing_gmID_list = green_route_gmID_list[ len( green_route_gmID_list) // 2 : ]

In [72]:
reference_green_route_best_pose_df = cfs.retrieve_gmID_topic( green_route_gmID_list[ 0 ], '/apollo/sensor/gnss/best/pose' )

#

green_route_training_combined_df_list = []

green_route_testing_combined_df_list = []

for gmID in green_route_gmID_list:

    current_gmID_chassis_df = cfs.retrieve_gmID_topic( gmID, '/apollo/canbus/chassis' )

    current_gmID_best_pose_df = cfs.retrieve_gmID_topic( gmID, '/apollo/sensor/gnss/best/pose' )

    #

    current_gmID_chassis_df = current_gmID_chassis_df.sort_values( 'time' )

    current_gmID_best_pose_df = current_gmID_best_pose_df.sort_values( 'time' )

    #

    cfs.BinaryDrivingMode( current_gmID_chassis_df )

    cfs.TernaryDrivingModeTransition( current_gmID_chassis_df )

    #

    cfs.LatLonTotalStdDev( current_gmID_best_pose_df )

    cfs.ProgressAlongRoute( current_gmID_best_pose_df, reference_green_route_best_pose_df )

    #

    cfs.ChassisBestPoseMatchedTime( current_gmID_chassis_df, current_gmID_best_pose_df )

    #

    current_gmID_combined_df = pd.merge( current_gmID_chassis_df, current_gmID_best_pose_df, on = 'ChassisBestPoseMatchedTime', how = 'inner' )

    if gmID in green_route_training_gmID_list:

        green_route_training_combined_df_list.append( current_gmID_combined_df )

    if gmID in green_route_testing_gmID_list:

        green_route_testing_combined_df_list.append( current_gmID_combined_df )

In [73]:
green_route_training_combined_df_all = pd.concat( green_route_training_combined_df_list )

green_route_testing_combined_df_all = pd.concat( green_route_testing_combined_df_list )

In [75]:
green_route_training_combined_df_all.shape

(1084952, 25)

In [76]:
green_route_testing_combined_df_all.shape

(1185592, 25)

In [77]:
d_cols = [ 'speedMps', 'brakePercentage', 'throttlePercentage', 'steeringPercentage', 'LatLonTotalStdDev', 'ProgressAlongRoute' ]

r_cols = [ 'BinaryDrivingMode' ]

#

X_green_train = green_route_training_combined_df_all[ d_cols ]

Y_green_train = green_route_training_combined_df_all[ r_cols ]

#

X_green_test = green_route_testing_combined_df_all[ d_cols ]

Y_green_test = green_route_testing_combined_df_all[ r_cols ]

In [78]:
clf = tree.DecisionTreeClassifier( max_depth = 7 )

clf = clf.fit( X_green_train, Y_green_train )

In [79]:
prediction = clf.predict( X_green_test )

In [80]:
results = X_green_test.copy()

results[ 'predicted_BinaryDrivingMode' ] = prediction

results[ 'actual_BinaryDrivingMode' ] = np.array( Y_green_test[ 'BinaryDrivingMode' ] )

In [81]:
correct = 0

incorrect = 0

for prediction, actual in zip( np.array( results[ 'predicted_TernaryDrivingModeTransition' ] ),
                               np.array( results[ 'actual_TernaryDrivingModeTransition' ] ) ):

    if ( prediction == actual ):

        correct = correct + 1

    elif ( prediction != actual ):

        incorrect = incorrect + 1

total = correct + incorrect

print( f'Correct: { correct }' )

print( f'Incorrect: { incorrect }' )

print( f'Total: { total }' )

Correct: 1040435
Incorrect: 145157
Total: 1185592


In [82]:
correct = 0

incorrect = 0

for prediction, actual in zip( np.array( results[ 'predicted_TernaryDrivingModeTransition' ] ),
                               np.array( results[ 'actual_TernaryDrivingModeTransition' ] ) ):

    if ( actual == 0 ):

        if ( prediction == actual ):

            correct = correct + 1

        elif ( prediction != actual ):

            incorrect = incorrect + 1

total = correct + incorrect

print( f'Correct: { correct }' )

print( f'Incorrect: { incorrect }' )

print( f'Total: { total }' )

Correct: 98198
Incorrect: 87896
Total: 186094
